# DATA SCIENCE ANALYSIS WITH PYTHON
# Midterm take-home

### Due 2/19/2018 11:59 pm

<p>Instructions
<ol>
<li>Download the files <i>medical_payment.csv</i> and <i>geo-data.csv</i> into the same folder
<li>Read the file <i>Medical_Payment-pdf.pdf</i>
<li>Run the code below and then answer the questions
</ol></p>

<b>Penalties:</b> You will incur penalties if:
<ul>
<li>Your answer is different from the correct one</li>
<li>Your code is unnecessarily slow</li>
<li>Your code is longer than specified</li>
<li>You will be penalized if, in an attempt to limit the lines of code, you make your code too hard to read or too slow -- for example, by copy-pasting pieces of code in the same line instead of declaring a variable in one line and using the variable in another.
</ul>

<b>Data Set:</b>https://drive.google.com/file/d/1HFIHsuQd7H5TuFLxubzb89_4t4pF5cO2/view?usp=sharing

In [209]:
import pandas as pd
import numpy as np 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

In [210]:
df = pd.read_csv("medical_payment.csv",dtype=str).\
        rename(columns=lambda x: x.strip())

## Data Description

With this data set, people can see the price difference for a hundred common illnesses and procedures. Hospitals can set their own prices and differ substantially in some cases.
 
Before you move on, read the <b>Medical_Payment-pdf.pdf</b> to understand the dataset.

In [211]:
df.shape

(163065, 10)

The columns are as follows:

In [212]:
df.columns

Index(['DRG Definition', 'Provider Id', 'Provider Name',
       'Provider Street Address', 'Provider Zip Code',
       'Hospital Referral Region Description', 'Total Discharges',
       'Average Covered Charges', 'Average Total Payments',
       'Average Medicare Payments'],
      dtype='object')

A sample of the dataset:

In [213]:
df.sample(5, random_state=1234).head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
30075,193 - SIMPLE PNEUMONIA & PLEURISY W MCC,210033,CARROLL HOSPITAL CENTER,200 MEMORIAL AVE,21157,MD - Baltimore,23,"$16,013.95","$15,102.30","$13,555.52"
53790,"280 - ACUTE MYOCARDIAL INFARCTION, DISCHARGED ...",150026,IU HEALTH GOSHEN HOSPITAL,200 HIGH PARK AVE,46526,IN - South Bend,11,"$34,427.00","$9,429.09","$8,711.63"
35917,195 - SIMPLE PNEUMONIA & PLEURISY W/O CC/MCC,260180,CHRISTIAN HOSPITAL NORTHEAST,11133 DUNN ROAD,63136,MO - St. Louis,26,"$16,288.38","$4,093.92","$3,312.69"
1627,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,230024,SINAI-GRACE HOSPITAL,6071 W OUTER DRIVE,48235,MI - Detroit,22,"$14,041.63","$8,260.90","$7,392.00"
143707,690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC,360112,MERCY ST VINCENT MEDICAL CENTER,2213 CHERRY STREET,43608,OH - Toledo,80,"$20,527.33","$6,425.51","$5,713.80"


## Q1. Transform relevant columns

All columns in the dataset are of type str. At least 4 of them should be numeric.

1.1 Identify them and convert the values of these columns to numeric.

In [214]:
type(df['Total Discharges'])
type(df['Provider Id'])
type(df['Provider Zip Code'])

pandas.core.series.Series

pandas.core.series.Series

pandas.core.series.Series

In [216]:
df['Total Discharges'] = df['Total Discharges'].astype(int)
df['Average Covered Charges'] = [float("".join(x.replace('$', '').split(","))) for x in df['Average Covered Charges']]
df['Average Total Payments'] = [float("".join(x.replace('$', '').split(","))) for x in df['Average Total Payments']]
df['Average Medicare Payments'] = [float("".join(x.replace('$', '').split(","))) for x in df['Average Medicare Payments']]

In [217]:
type(df['Average Medicare Payments'])

pandas.core.series.Series

1.2 Explain why these values should be numeric.

In [441]:
#Consider the following example:
str('1')+str('2')
int(1)+int(2)

'12'

3

since the values in total discharges ,Average Covered Charges, Average Total Payments, Average Medicare Payments are <b>numeric values</b> the type <b>string</b> will make the job very difficult to do any manipulations or data calculation in order to do analysis

## Q2. Let's take a look at heart disease

**Be careful when you identify 'heart' disease.** There are more than one type of DRG definition related to heart diseases. Make sure you catch all of them.  (hint 1:  use .unique() to start your investigation. hint 2: there are two keywords can be used to find all 'heart' diseases.

2.1, Total how many heart related disease cases across the whole country ?

In [218]:
df['heart']=(df['DRG Definition'].str.contains('HEART'))| (df['DRG Definition'].str.contains('CARDI'))



In [219]:
df.loc[df['heart'],'Total Discharges'].sum()

1189090

2.2, Create an overview from the view point of 'Hospital Referral Region Description' and 'Average Total Payments', sorted by payments in descending order. List the top 10 only.

In [173]:
df[(df['heart']== True)].groupby('Hospital Referral Region Description')['Average Total Payments'].agg(
{'Average Total Payments':'max'}).sort_values(by='Average Total Payments',ascending=False).head(10)
        

,Average Total Payments
Hospital Referral Region Description,
WA - Seattle,84499.26
SD - Sioux Falls,83484.14
MD - Baltimore,54682.48
NJ - Camden,54231.54
TX - Houston,54004.36
IL - Chicago,48218.21
CA - Salinas,44286.90
IL - Hinsdale,43939.25
IN - Indianapolis,43834.54


2.3, Find out which hospital treat heart disease in "CA - Alameda County" region, sorted by the total case number they handle by descending order.

In [327]:
df[(df['heart']== True) & (df['Hospital Referral Region Description']=='CA - Alameda County')].groupby('Provider Name')['Total Discharges'].agg(
{'Total Discharges':'sum'}).sort_values(by='Total Discharges',ascending=False)
        

,Total Discharges
Provider Name,
ALTA BATES SUMMIT MEDICAL CENTER,658
WASHINGTON HOSPITAL,430
EDEN MEDICAL CENTER,400
ALTA BATES SUMMIT MEDICAL CENTER - ALTA BATES CAMP,297
ST ROSE HOSPITAL,264
VALLEYCARE MEDICAL CENTER,197
ALAMEDA HOSPITAL,118
KAISER FOUNDATION HOSPITAL OAKLAND/RICHMOND,70
ALAMEDA COUNTY MEDICAL CENTER,62


## Q3
Merge the data set with mapping of US Zipcode, county, and state information from Census data.
Read the file `geo-data.csv` into an object `zip_codes`(make sure that you read all columns as string). After merged, there may have some columns (state and state_abbr) have nan values, fill them in with the information you can find in the first dataframe.

3.1 Based on the merged dataframe, find out the total DRG number for each city in California's Santa Clara county.

In [175]:
zip_codes = pd.read_csv("geo-data.csv",dtype=str)

In [176]:
zip_codes.dtypes

state_fips    object
state         object
state_abbr    object
zipcode       object
county        object
city          object
dtype: object

In [196]:
zip_codes.head(5)


,state_fips,state,state_abbr,Provider Zip Code,county,city
0,1,Alabama,AL,35004,St. Clair,Acmar
1,1,Alabama,AL,35005,Jefferson,Adamsville
2,1,Alabama,AL,35006,Jefferson,Adger
3,1,Alabama,AL,35007,Shelby,Keystone
4,1,Alabama,AL,35010,Tallapoosa,New site


In [178]:
df.head(3)


,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,heart
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,36301,AL - Dothan,91,32963.07,5777.24,4763.73,False
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,35957,AL - Birmingham,14,15131.85,5787.57,4976.71,False
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,35631,AL - Birmingham,24,37560.37,5434.95,4453.79,False


In [179]:
zip_codes = zip_codes.rename(columns={'zipcode': 'Provider Zip Code' })

In [181]:
zip_codes.head()
df.head(5)


,state_fips,state,state_abbr,Provider Zip Code,county,city
0,1,Alabama,AL,35004,St. Clair,Acmar
1,1,Alabama,AL,35005,Jefferson,Adamsville
2,1,Alabama,AL,35006,Jefferson,Adger
3,1,Alabama,AL,35007,Shelby,Keystone
4,1,Alabama,AL,35010,Tallapoosa,New site


,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,heart
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,36301,AL - Dothan,91,32963.07,5777.24,4763.73,False
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,35957,AL - Birmingham,14,15131.85,5787.57,4976.71,False
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,35631,AL - Birmingham,24,37560.37,5434.95,4453.79,False
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,35235,AL - Birmingham,25,13998.28,5417.56,4129.16,False
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,35007,AL - Birmingham,18,31633.27,5658.33,4851.44,False


In [182]:
df2 = df.merge(zip_codes,how='left')



In [184]:
df2.isnull().sum()


DRG Definition                              0
Provider Id                                 0
Provider Name                               0
Provider Street Address                     0
Provider Zip Code                           0
Hospital Referral Region Description        0
Total Discharges                            0
Average Covered Charges                     0
Average Total Payments                      0
Average Medicare Payments                   0
heart                                       0
state_fips                              14262
state                                   14262
state_abbr                              14262
county                                  14262
city                                    14363
dtype: int64

In [186]:
df2.loc[df2['state_abbr'].isnull(),'state_abbr']=df2['Hospital Referral Region Description'].str.slice(0,2)

In [188]:
df2['state']=df2.groupby('state_abbr')['state'].fillna(method='ffill')
df2['state']=df2.groupby('state_abbr')['state'].fillna(method='bfill')

In [191]:
df2.head(5)

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,heart,state_fips,state,state_abbr,county,city
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,36301,AL - Dothan,91,32963.07,5777.24,4763.73,False,1,Alabama,AL,Houston,Taylor
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,35957,AL - Birmingham,14,15131.85,5787.57,4976.71,False,1,Alabama,AL,Marshall,Boaz
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,35631,AL - Birmingham,24,37560.37,5434.95,4453.79,False,NaN,Alabama,AL,NaN,NaN
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,35235,AL - Birmingham,25,13998.28,5417.56,4129.16,False,1,Alabama,AL,Jefferson,Center point
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,35007,AL - Birmingham,18,31633.27,5658.33,4851.44,False,1,Alabama,AL,Shelby,Keystone


In [220]:
df2[(df2['county']== 'Santa Clara') & (df2['state']== 'California') ].groupby('city')['DRG Definition'].size()

city
Gilroy            31
Mountain view     81
San jose         285
Santa clara        8
Stanford          87
Name: DRG Definition, dtype: int64

## Q4
4.1 Identify five hospitals and show the hospital name, city and state information that offer lowest spending on lung disease treatment 

hint 1: there are two keywords can be used to find all 'lung' diseases. 

hint 2: do not include any RESPIRATORY SYSTEM diseases.

In [222]:
df2.head(5)

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,heart,state_fips,state,state_abbr,county,city,lung
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,36301,AL - Dothan,91,32963.07,5777.24,4763.73,False,1,Alabama,AL,Houston,Taylor,False
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,35957,AL - Birmingham,14,15131.85,5787.57,4976.71,False,1,Alabama,AL,Marshall,Boaz,False
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,35631,AL - Birmingham,24,37560.37,5434.95,4453.79,False,NaN,Alabama,AL,NaN,NaN,False
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,35235,AL - Birmingham,25,13998.28,5417.56,4129.16,False,1,Alabama,AL,Jefferson,Center point,False
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,35007,AL - Birmingham,18,31633.27,5658.33,4851.44,False,1,Alabama,AL,Shelby,Keystone,False


In [227]:
df2['lung']=(df['DRG Definition'].str.contains('PULMONARY'))| (df['DRG Definition'].str.contains('PNEUMONIA'))

In [245]:
df2.loc[(df2['lung']== True),['Provider Name','Average Total Payments','city','state']].sort_values(
    by='Average Total Payments',ascending=True).head(5)



,Provider Name,Average Total Payments,city,state
35193,HART COUNTY HOSPITAL,3338.17,Hartwell,Georgia
35515,VIA CHRISTI HOSPITAL WICHITA ST TERESA INC,3395.00,Wichita,Kansas
34674,SPRINGHILL MEDICAL CENTER,3444.00,Mobile,Alabama
27296,VIA CHRISTI HOSPITAL WICHITA ST TERESA INC,3455.00,Wichita,Kansas
27233,KENTUCKIANA MEDICAL CENTER LLC,3468.48,Clarksville,Indiana


## Q5

For a patient who has "misc disorders of nutrition,metabolism,fluids/electrolytes", which hospital across the whole United States will be able to offer the lowest cost from average total payments point of view.

5.1 Please find that hospital.

In [333]:
df2['Misc']=(df['DRG Definition'].str.contains('MISC DISORDERS OF NUTRITION,METABOLISM,FLUIDS/ELECTROLYTES'))
                                               

In [335]:
df2.loc[(df2['Misc']== True),['Provider Name','Average Total Payments']].sort_values(
    by='Average Total Payments',ascending=True).head(1)



,Provider Name,Average Total Payments
134051,WEIRTON MEDICAL CENTER,3286.11


5.2 From 'Average Covered Charges' point of view, What's the ratio between the city with the highest average and the lowest average ?
What are those two cities ?

In [363]:
dfmin=df2.loc[(df2['Misc']== True),['Provider Name','city','Average Covered Charges']].sort_values(
    by='Average Covered Charges',ascending=True).head(1)

In [364]:
# city with minium Average Covered Charges
dfmin

,Provider Name,city,Average Covered Charges
133883,COMMUNITY GENERAL HOSPITAL,Dilley,2995.61


In [365]:
dfmax=df2.loc[(df2['Misc']== True),['Provider Name','city','Average Covered Charges']].sort_values(
    by='Average Covered Charges',ascending=False).head(1)

In [366]:
# city with maximum Average Covered Charges
dfmax

,Provider Name,city,Average Covered Charges
129667,STANFORD HOSPITAL,Stanford,144497.2


In [367]:
dfmax['Average Covered Charges'].values[0]
dfmin['Average Covered Charges'].values[0]

144497.2

2995.61

In [368]:
new = dfmax['Average Covered Charges'].values[0]/dfmin['Average Covered Charges'].values[0]

In [381]:

print( 'The ratio of  Average Covered Charges  between the highest and the lowest is %f' %new )

The ratio of  Average Covered Charges  between the highest and the lowest is 48.236319


## Q6 

6.1 For California, which disease is the most costly one?

In [352]:
df2[(df2['state']== 'California') ].groupby('DRG Definition')['Average Total Payments'].agg(
{'Average Total Payments':'max'}).sort_values(by='Average Total Payments',ascending=False).head((1))
        

,Average Total Payments
DRG Definition,
207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR SUPPORT 96+ HOURS,156158.18


6.2 For each state, find the disease that is the most costly one

In [399]:
df2.groupby('state').apply(lambda d: d[d['Average Total Payments'] == d['Average Total Payments'].max()])[
    ['DRG Definition','Average Total Payments']]

,,DRG Definition,Average Total Payments
state,,,
Alabama,151372,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,48198.38
Alaska,86932,329 - MAJOR SMALL & LARGE BOWEL PROCEDURES W MCC,77566.38
Arizona,150016,853 - INFECTIOUS & PARASITIC DISEASES W O.R. P...,71437.35
Arkansas,86970,329 - MAJOR SMALL & LARGE BOWEL PROCEDURES W MCC,57325.95
California,39594,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,156158.18
Colorado,151529,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,68294.66
Connecticut,151537,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,73887.44
Delaware,150234,853 - INFECTIOUS & PARASITIC DISEASES W O.R. P...,50707.91
District of columbia,150236,853 - INFECTIOUS & PARASITIC DISEASES W O.R. P...,99067.50


6.3 Based on the above results, provide your analysis.

<b>In order to do a analysis lets do minor Manipulations</b>

In [403]:
df2.groupby('state').apply(lambda d: d[d['Average Total Payments'] == d['Average Total Payments'].max()])[
    ['DRG Definition','Average Total Payments']].sort_values(by='Average Total Payments',ascending=False).head(5)

,,DRG Definition,Average Total Payments
state,,,
California,39594,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,156158.18
New york,150886,853 - INFECTIOUS & PARASITIC DISEASES W O.R. P...,140255.26
Indiana,109526,460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC,131187.35
Pennsylvania,111140,469 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...,100018.33
South carolina,88111,329 - MAJOR SMALL & LARGE BOWEL PROCEDURES W MCC,99307.30


So the top 5 states with the highest <b>Average Total Payments</b> are CA, NY, IN, PA and SC of which 
<b> California </b> has the maximum <b>Average Total Payments</b> cost with most expensive treatment for Diseases  realted to <b>RESPIRATORY SYSTEM</b>

In [407]:
df2.groupby('state').apply(lambda d: d[d['Average Total Payments'] == d['Average Total Payments'].max()])[
    ['DRG Definition','Average Total Payments']].sort_values(by='Average Total Payments',ascending=False).tail(5)

,,DRG Definition,Average Total Payments
state,,,
Tennessee,152165,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,48753.13
Alabama,151372,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,48198.38
Maine,151749,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,45990.20
Montana,150747,853 - INFECTIOUS & PARASITIC DISEASES W O.R. P...,40562.45
North dakota,152042,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,39845.68


So the top 5 states with the lowes <b>Average Total Payments</b> are TN, AL, ME, MT and ND of which 
<b> North dakota </b> has the Minimum  <b>Average Total Payments</b> cost with most expensive treatment for Diseases  realted to <b>SEPTICEMIA OR SEVERE SEPSIS</b>

Another major thing to notice is that if <b>SEPTICEMIA OR SEVERE SEPSIS</b> is the most expensive disease 
than the cost of treating other Diseases is not great compared to other states in <b>USA</b> and 
hence the insurance cost are very low in <b>TN, AL, ME, MT and ND</b>


## Q7
Based on the merged data set, describe the most interesting or unexpected finding you may have. (note: you need to list the related code to back up/prove your statement.

Lets use corr() function to see if there are any relations!



In [434]:
df2.describe()

,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
count,163166.000000,163166.000000,163166.000000,163166.000000
mean,42.763358,36123.429023,9706.502557,8493.630521
std,51.092548,35058.543111,7663.468371,7308.194954
min,11.000000,2459.400000,2673.000000,1148.900000
25%,17.000000,15940.502500,5234.737500,4192.625000
50%,27.000000,25238.185000,7213.805000,6158.330000
75%,49.000000,43219.382500,11284.242500,10055.970000
max,3383.000000,929118.900000,156158.180000,154620.810000


In [323]:
cor =df2.corr()
cor

,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,heart
Total Discharges,1.000000,-0.020622,-0.015942,-0.021225,0.016023
Average Covered Charges,-0.020622,1.000000,0.774100,0.768918,0.029541
Average Total Payments,-0.015942,0.774100,1.000000,0.989363,0.011061
Average Medicare Payments,-0.021225,0.768918,0.989363,1.000000,0.010100
heart,0.016023,0.029541,0.011061,0.010100,1.000000


In [324]:
cor.stack()

Total Discharges           Total Discharges             1.000000
                           Average Covered Charges     -0.020622
                           Average Total Payments      -0.015942
                           Average Medicare Payments   -0.021225
                           heart                        0.016023
Average Covered Charges    Total Discharges            -0.020622
                           Average Covered Charges      1.000000
                           Average Total Payments       0.774100
                           Average Medicare Payments    0.768918
                           heart                        0.029541
Average Total Payments     Total Discharges            -0.015942
                           Average Covered Charges      0.774100
                           Average Total Payments       1.000000
                           Average Medicare Payments    0.989363
                           heart                        0.011061
Average Medicare Payments

In [325]:
cor[cor<1].stack().nlargest(20)[::2]

Average Total Payments     Average Medicare Payments    0.989363
Average Covered Charges    Average Total Payments       0.774100
Average Medicare Payments  Average Covered Charges      0.768918
heart                      Average Covered Charges      0.029541
Total Discharges           heart                        0.016023
heart                      Average Total Payments       0.011061
Average Medicare Payments  heart                        0.010100
Total Discharges           Average Total Payments      -0.015942
Average Covered Charges    Total Discharges            -0.020622
Average Medicare Payments  Total Discharges            -0.021225
dtype: float64

In [326]:
cor[cor<1].stack().nsmallest(20)[::2]

Total Discharges           Average Medicare Payments   -0.021225
                           Average Covered Charges     -0.020622
                           Average Total Payments      -0.015942
Average Medicare Payments  heart                        0.010100
heart                      Average Total Payments       0.011061
Total Discharges           heart                        0.016023
heart                      Average Covered Charges      0.029541
Average Covered Charges    Average Medicare Payments    0.768918
                           Average Total Payments       0.774100
Average Total Payments     Average Medicare Payments    0.989363
dtype: float64

so after using the function we can find that there is no significant realtion!!

In [419]:
df2.groupby('state').apply(lambda d: d.groupby('DRG Definition').size().nlargest(1))

state                 DRG Definition                                                          
Alabama               690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC                              88
Alaska                194 - SIMPLE PNEUMONIA & PLEURISY W CC                                        8
Arizona               603 - CELLULITIS W/O MCC                                                     54
Arkansas              194 - SIMPLE PNEUMONIA & PLEURISY W CC                                       44
California            871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W MCC                    273
Colorado              470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC     44
Connecticut           177 - RESPIRATORY INFECTIONS & INFLAMMATIONS W MCC                           31
Delaware              189 - PULMONARY EDEMA & RESPIRATORY FAILURE                                   6
District of columbia  065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W CC          

In [429]:
df2.groupby('state').apply(lambda d:d[d['Total Discharges']== d['Total Discharges'].max()])[['DRG Definition','Total Discharges','Average Total Payments']].sort_values(
    by='Total Discharges',ascending=False).head(5) 


,,DRG Definition,Total Discharges,Average Total Payments
state,,,,
New york,113035,470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...,3383,19023.70
California,157680,885 - PSYCHOSES,1696,8192.84
Massachusetts,158617,897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REH...,1571,4967.56
Illinois,155669,885 - PSYCHOSES,1487,8165.69
Florida,102299,"392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DIS...",1344,4727.96


So I have tried to classify the most expensive disease in a State and the number of patients treated by the disease 
and we can see that <b> New york</b> has the highest number of cases of <b>MAJOR JOINT REPLACEMENT </b> 
where the cost of treatment is around <b>19023.70</b>


In [436]:
new = df2.groupby('state').apply(lambda d: d[d['Average Total Payments'] == d['Average Total Payments'].max()])[['DRG Definition','Average Total Payments']]

In [437]:
new[new['DRG Definition'].str.contains('INFECTIOUS')].count()

DRG Definition            18
Average Total Payments    18
dtype: int64

In [438]:
new[new['DRG Definition'].str.contains('SEPTICEMIA')].count()

DRG Definition            17
Average Total Payments    17
dtype: int64

It is interesting to see that <b>35</b> out <b>51</b> States in the <b>US</b> spend the most money to treat 
<b>Infectious Diseases</b> which is about <b>70%</b> of the total number of states in the <b>US </b>